In [132]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from math import log, cos, sin
from numpy import exp

In [303]:
class DE:
    def y_p(self,x,y,y_prim):
        return eval(y_prim)
    def __init__(self,y_prime = "exp(y)-2/x", x_init = 1 , y_init = -2, X = 10, h = 1):
        self.x_init = x_init
        assert self.x_init > 0, "x should bigger than 0" + str(self.x_init)
        self.y_init = y_init
        self.X = X
        assert self.X > self.x_init, "X should bigger than x_int"
        self.h = h
        assert self.h > 0, "h should bigger than 0"
        self.y_prime = y_prime
        self.C = 1/self.x_init - 1/(exp(self.y_init)*self.x_init**2)
        self.x_arr = np.arange(self.x_init,self.X,self.h)

class Analytical_Method(DE):
    def y(self,x,C,y_fun):
        return eval(y_fun)
    def __init__(self,y_prime = "exp(y)-2/x", x_init = 1 , y_init = -2, X = 10, h = 1, y_func = "log(1/(x-C*x**2))"):
        DE.__init__(self,y_prime, x_init , y_init , X, h)
        self.y_func = y_func
        self.y_arr = np.array([self.y_init])
    def solve(self):
        self.y_arr = np.array([self.y_init])
        for i in range(1,self.x_arr.size):
            self.y_arr = np.append(self.y_arr, self.y(self.x_arr[i],self.C, self.y_func))
        return  self.y_arr

class Numerical_Method(DE):
    def __init__(self,y_prime = "exp(y)-2/x", x_init = 1 , y_init = -2, X = 10, h = 1):
        DE.__init__(self,y_prime, x_init , y_init , X, h)
        self.y_arr = np.array([self.y_init])

class Euler_Method(Numerical_Method):
    def __init__(self,y_prime = "exp(y)-2/x", x_init = 1 , y_init = -2, X = 10, h = 1):
        Numerical_Method.__init__(self,y_prime, x_init , y_init , X, h)
    def solve(self):
        self.y_arr = np.array([self.y_init])
        for i in range(self.x_arr.size-1):
            self.y_arr = np.append(self.y_arr,self.y_arr[i] + self.h*self.y_p(self.x_arr[i], self.y_arr[i],self.y_prime))
        return self.y_arr
    def solve_LTE(self,y_ex):
        self.y_arr = np.array([self.y_init])
        for i in range(self.x_arr.size-1):
            self.y_arr = np.append(self.y_arr,y_ex[i] + self.h*self.y_p(self.x_arr[i], y_ex[i],self.y_prime))
        return self.y_arr

class Improved_Euler_Method(Numerical_Method):
    def __init__(self,y_prime = "exp(y)-2/x", x_init = 1 , y_init = -2, X = 10, h = 1):
        Numerical_Method.__init__(self,y_prime, x_init , y_init , X, h)
    def IEM(self,x,y,h):
        return self.y_p(x,y,self.y_prime) + self.y_p(x+h,y + h*self.y_p(x,y,self.y_prime),self.y_prime)
    def solve(self):
        self.y_arr = np.array([self.y_init])
        for i in range(self.x_arr.size-1):
            self.y_arr = np.append(self.y_arr,self.y_arr[i] + self.h/2*self.IEM(self.x_arr[i],self.y_arr[i],self.h))
        return self.y_arr
    def solve_LTE(self,y_ex):
        self.y_arr = np.array([self.y_init])
        for i in range(self.x_arr.size-1):
            self.y_arr = np.append(self.y_arr,y_ex[i] + self.h/2*self.IEM(self.x_arr[i],y_ex[i],self.h))
        return self.y_arr

class Runge_Kutta(Numerical_Method):
    def RK(self,x,y,h):
        k1 = self.y_p(x,y,self.y_prime)
        k2 = self.y_p(x + h/2,y+h*k1/2,self.y_prime)
        k3 = self.y_p(x + h/2, y + h*k2/2,self.y_prime)
        k4 = self.y_p(x + h,y + h*k3,self.y_prime)
        return (k1 + 2*k2 + 2*k3 + k4)
    def __init__(self,y_prime = "exp(y)-2/x", x_init = 1 , y_init = -2, X = 10, h = 1):
        Numerical_Method.__init__(self,y_prime, x_init , y_init , X, h)
    def solve(self):
        self.y_arr = np.array([self.y_init])
        for i in range(self.x_arr.size-1):
            self.y_arr = np.append(self.y_arr,self.y_arr[i] + self.h/6*self.RK(self.x_arr[i],self.y_arr[i],self.h))
        return self.y_arr
    def solve_LTE(self, y_ex):
        self.y_arr = np.array([self.y_init])
        for i in range(self.x_arr.size-1):
            self.y_arr = np.append(self.y_arr,y_ex[i] + self.h/6*self.RK(self.x_arr[i],y_ex[i],self.h))
        return self.y_arr

class Grid(DE):
    def __init__(self,y_prime = "exp(y)-2/x", x_init = 1 , y_init = -2, X = 10, h = 1, check = "1111"):
        DE.__init__(self,y_prime, x_init , y_init , X, h)
        self.check = check
        self.em = Euler_Method(y_prime, x_init,y_init,X,h)
        self.iem = Improved_Euler_Method(y_prime, x_init,y_init,X,h)
        self.rk = Runge_Kutta(y_prime, x_init,y_init,X,h)
        self.am = Analytical_Method(y_prime, x_init,y_init,X,h)

    def plot_functions(self):
        self.ys = np.array([])
        self.names = np.array([])
        if self.check[0] == "1" :
            self.ys = np.append(self.ys, self.em.solve())
            self.names = np.append(self.names, "Euler Method")
        if self.check[1] == '1' :
            self.ys = np.append(self.ys, self.iem.solve())
            self.names = np.append(self.names, "Improved Euler Method")
        if self.check[2] == '1' :
            self.ys = np.append(self.ys, self.rk.solve())
            self.names = np.append(self.names, "Runge Kutta")
        if self.check[3] == '1' :
            self.ys = np.append(self.ys, self.am.solve())
            self.names = np.append(self.names, "Analytical Method")
        if self.names.size > 0 : self.ys = np.array_split(self.ys, self.names.size)
        plt.title("graphs for h = {0}, a = {1}, b = {2}".format(self.h, self.x_init, self.X), fontsize = 20,color = 'black')
        for i in range(self.names.size):
            plt.plot(self.x_arr,self.ys[i],label = self.names[i])
        if self.names.size > 0 : plt.legend()
        plt.grid()
        return plt.show()

    def GTE(self, y_ap, y_ex):
        return abs(y_ap - y_ex)
    def LTE(self, y_ap, y_ex):
        return abs(y_ap - y_ex)
    def plot_GTE(self):
        self.y_am = self.am.solve()
        self.ys_err = np.array([])
        self.names = np.array([])
        if self.check[0] == "1" :
            self.ys_err = np.append(self.ys_err, self.GTE(self.em.solve(), self.y_am))
            self.names = np.append(self.names, "Euler Method")
        if self.check[1] == '1' :
            self.ys_err = np.append(self.ys_err, self.GTE(self.iem.solve(), self.y_am))
            self.names = np.append(self.names, "Improved Euler Method")
        if self.check[2] == '1' :
            self.ys_err = np.append(self.ys_err, self.GTE(self.rk.solve(), self.y_am))
            self.names = np.append(self.names, "Runge Kutta")
        if self.check[3] == '1' :
            self.ys_err = np.append(self.ys_err, self.am.solve() - self.y_am)
            self.names = np.append(self.names, "Analytical Method")
        if self.names.size > 0 : self.ys_err = np.array_split(self.ys_err, self.names.size)
        plt.title("GTE for h = {0}, a = {1}, b = {2}".format(self.h, self.x_init, self.X), fontsize = 20,color = 'black')
        for i in range(self.names.size):
            plt.plot(self.x_arr,self.ys_err[i],label = self.names[i])
        if self.names.size > 0 : plt.legend()
        plt.grid()
        return plt.show()

    def plot_LTE(self):
        self.y_am = self.am.solve()
        self.ys_err = np.array([])
        self.names = np.array([])
        if self.check[0] == "1" :
            self.ys_err = np.append(self.ys_err, self.LTE(self.em.solve_LTE(self.y_am), self.y_am))
            self.names = np.append(self.names, "Euler Method")
        if self.check[1] == '1' :
            self.ys_err = np.append(self.ys_err, self.LTE(self.iem.solve_LTE(self.y_am), self.y_am))
            self.names = np.append(self.names, "Improved Euler Method")
        if self.check[2] == '1' :
            self.ys_err = np.append(self.ys_err, self.LTE(self.rk.solve_LTE(self.y_am), self.y_am))
            self.names = np.append(self.names, "Runge Kutta")
        if self.check[3] == '1' :
            self.ys_err = np.append(self.ys_err, self.am.solve() - self.y_am)
            self.names = np.append(self.names, "Analytical Method")
        if self.names.size > 0 : self.ys_err = np.array_split(self.ys_err, self.names.size)
        plt.title("LTE for h = {0}, a = {1}, b = {2}".format(self.h, self.x_init, self.X), fontsize = 20,color = 'black')
        for i in range(self.names.size):
            plt.plot(self.x_arr,self.ys_err[i],label = self.names[i])
        if self.names.size > 0 : plt.legend()
        plt.grid()
        return  plt.show()

    def plot_max_GTE(self, H = 5, step = 0.1):
        assert H > self.h , "H should be bigger then h"
        assert H < self.X , "H should be less then X"
        assert step > 0, "step should be bigger then 0"
        self.names = np.array([])
        if self.check[0] == "1" :
            self.names = np.append(self.names, "Euler Method")
        if self.check[1] == "1" :       
            self.names = np.append(self.names, "Improved Euler Method")
        if self.check[2] == "1" :
            self.names = np.append(self.names, "Runge Kutta")
        self.max_errs = []
        self.h_arr = np.arange(self.h, H+step, step)
        for i in self.h_arr:
            temp = Grid(self.y_prime, self.x_init, self.y_init, self.X, i, self.check)
            self.y_am = temp.am.solve()
            if self.check[0] == "1" :
                self.max_errs.append(max(self.GTE(temp.em.solve(), self.y_am)))
            if self.check[1] == "1" :
                self.max_errs.append(max(self.GTE(temp.iem.solve(), self.y_am)))
            if self.check[2] == "1" :
                self.max_errs.append(max(self.GTE(temp.rk.solve(), self.y_am)))
        if self.names.size > 0 : self.max_errs = np.transpose(np.array_split(self.max_errs, self.max_errs.__len__()/self.names.size))
        plt.title("GTE for h = {0},H = {1}, step ={4}, a = {2}, b = {3}".format(self.h, H, self.x_init, self.X,step), fontsize = 20,color = 'black')
        for i in range(self.names.size):
            plt.plot(self.h_arr, self.max_errs[i], label = self.names[i])
        if self.names.size > 0 : plt.legend()
        plt.grid()
        plt.xlabel("step size")
        plt.ylabel("max error")
        return plt.show()

    def plot_max_LTE(self, H = 5, step = 0.1):
        assert H > self.h , "H should be bigger then h"
        assert H < self.X , "H should be less then X"
        assert step > 0, "step should be bigger then 0"
        self.names = np.array([])
        if self.check[0] == "1" :
            self.names = np.append(self.names, "Euler Method")
        if self.check[1] == "1" :
            self.names = np.append(self.names, "Improved Euler Method")
        if self.check[2] == "1" :
            self.names = np.append(self.names, "Runge Kutta")
        self.max_errs = []
        self.h_arr = np.arange(self.h, H+step, step)
        for i in self.h_arr:
            temp = Grid(self.y_prime, self.x_init, self.y_init, self.X, i, self.check)
            self.y_am = temp.am.solve()
            if self.check[0] == "1" :
                self.max_errs.append(max(self.LTE(temp.em.solve_LTE(self.y_am), self.y_am)))
            if self.check[1] == "1" :
                self.max_errs.append(max(self.LTE(temp.iem.solve_LTE(self.y_am), self.y_am)))
            if self.check[2] == "1" :
                self.max_errs.append(max(self.LTE(temp.rk.solve_LTE(self.y_am), self.y_am)))
        if self.names.size > 0 : self.max_errs = np.transpose(np.array_split(self.max_errs, self.max_errs.__len__()/self.names.size))
        plt.title("LTE for h = {0},H = {1}, step ={4}, a = {2}, b = {3}".format(self.h, H, self.x_init, self.X,step), fontsize = 20,color = 'black')
        for i in range(self.names.size):
            plt.plot(self.h_arr, self.max_errs[i], label = self.names[i])
        if self.names.size > 0 : plt.legend()
        plt.grid()
        plt.xlabel("step size")
        plt.ylabel("max error")
        return plt.show()


In [300]:
"""def f_eu(x, y, h): #euler
    return exp(y) - 2/x
def f_eu_mod(x,y,h):#modified Euler
    return (f_eu(x, y, h) + f_eu(x+h,y + h*f_eu(x,y,h),h))
def f(x,C):   #init function
    return log(1/(x-C*x**2)) #x*(1+x**2/3)/(1-x**2/3)
def Runge_Kutta(x,y,h):
    k1 = f_eu(x,y,h)
    k2 = f_eu(x + h/2,y+h*k1/2,h)
    k3 = f_eu(x + h/2, y + h*k2/2,h)
    k4 = f_eu(x + h,y + h*k3,h)
    return (k1 + 2*k2 + 2*k3 + k4)
def find_const(x,y):
    return 1/x - 1/(exp(y)*x**2)


#input section
h = 1 #size of step
y_input = -2 #init y
x_input = 1 #init x
a = x_input #beginning
b = 10 #ending
#end of input section

C = find_const(x_input, y_input)
x_i = [x_input] # x
y_eu = [y_input] # approximate y with Euler
y_eu_mod = [y_input] # approximate y with modified Euler
y_RK =[y_input]

y = []  # y from init function
y_eu_err = [0] # errors of Euler method
y_eu_mod_err = [0] #errors of modified Euler method
y_RK_err =[0]
y_eu_LTE = [0] # errors of Euler method
y_eu_mod_LTE = [0] #errors of modified Euler method
y_RK_LTE =[0]


z = int((b-a)/h)
for i in range(z):
    y_eu.append(y_eu[i] + h*f_eu(x_i[i],y_eu[i],h))
    y_eu_mod.append(y_eu_mod[i] + h/2*f_eu_mod(x_i[i],y_eu_mod[i],h))
    y_RK.append(y_RK[i]+ h/6*Runge_Kutta(x_i[i],y_RK[i],h))
    x_i.append(x_i[i]+h)

for i in range(len(x_i)):
    y.append(f(x_i[i],C))
    if i > 0:
        y_eu_err.append(y[i] -  y_eu[i])
        y_eu_mod_err.append(y[i] - y_eu_mod[i])
        y_RK_err.append(y[i] - y_RK[i])
        y_eu_LTE.append(y[i] - y[i-1] - h*f_eu(x_i[i-1],y[i-1],h))
        y_eu_mod_LTE.append(y[i] - y[i-1] - h/2*f_eu_mod(x_i[i-1],y[i-1],h))
        y_RK_LTE.append(y[i] - y[i-1] - h/6*Runge_Kutta(x_i[i-1],y[i-1],h))

plt.title("h = {0}, a = {1}, b = {2}".format(h, a, b), fontsize = 20,color = 'black')
plt.plot(x_i,y_eu, label = 'Euler method')
plt.plot(x_i,y_eu_mod, label = 'Euler updated method')
plt.plot(x_i,y_RK, label = 'RK')
plt.plot(x_i,y, label = 'init')
plt.legend()
plt.savefig("function_plot.png")
plt.clf()

plt.title("Errors for h = {0}, a = {1}, b = {2}".format(h, a, b), fontsize = 20,color = 'black')
plt.plot(x_i,y_eu_err, label = 'Euler method')
plt.plot(x_i,y_eu_mod_err, label = 'Euler updated method')
plt.plot(x_i,y_RK_err, label = 'RK')
plt.legend()
plt.savefig("error_plot.png")

table = pd.DataFrame()
table["x"] = np.array(x_i)
table["y"] = np.array(y)
table["Euler"] = np.array(y_eu)
table["Euler method mod"] = np.array(y_eu_mod)
table["Runge-Kutta"] = np.array(y_RK)
table["Euler error"] = np.array(y_eu_err)
table["Euler mod error"] = np.array(y_eu_mod_err)
table["Rk error"] = np.array(y_RK_err)
table["Euler error LTE"] = np.array(y_eu_LTE)
table["Euler mod error LTE"] = np.array(y_eu_mod_LTE)
table["Rk error LTE"] = np.array(y_RK_LTE)
table.set_index("x",inplace=True)
table"""

'def f_eu(x, y, h): #euler\n    return exp(y) - 2/x\ndef f_eu_mod(x,y,h):#modified Euler\n    return (f_eu(x, y, h) + f_eu(x+h,y + h*f_eu(x,y,h),h))\ndef f(x,C):   #init function\n    return log(1/(x-C*x**2)) #x*(1+x**2/3)/(1-x**2/3)\ndef Runge_Kutta(x,y,h):\n    k1 = f_eu(x,y,h)\n    k2 = f_eu(x + h/2,y+h*k1/2,h)\n    k3 = f_eu(x + h/2, y + h*k2/2,h)\n    k4 = f_eu(x + h,y + h*k3,h)\n    return (k1 + 2*k2 + 2*k3 + k4)\ndef find_const(x,y):\n    return 1/x - 1/(exp(y)*x**2)\n\n\n#input section\nh = 1 #size of step\ny_input = -2 #init y\nx_input = 1 #init x\na = x_input #beginning\nb = 10 #ending\n#end of input section\n\nC = find_const(x_input, y_input)\nx_i = [x_input] # x\ny_eu = [y_input] # approximate y with Euler\ny_eu_mod = [y_input] # approximate y with modified Euler\ny_RK =[y_input]\n\ny = []  # y from init function\ny_eu_err = [0] # errors of Euler method\ny_eu_mod_err = [0] #errors of modified Euler method\ny_RK_err =[0]\ny_eu_LTE = [0] # errors of Euler method\ny_eu_mod_LTE